In [424]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [425]:
#load all ti matches
matches_df = pd.read_json('all_matches.json')
matches_df['match_id'].count()



149

In [426]:
# determine winner based on match_id
def get_winner(matchid):
    match_data = matches_df[matches_df['match_id']==matchid]
    radiant_win = match_data['radiant_win'].to_string(index=False)
    winner = 1
    if radiant_win == 'True':
        winner = 0
    return winner



In [427]:
#define heroes df
heroes_df = pd.read_json('heroes.json')
heroes_df = heroes_df[['id','localized_name']]
heroes_df.rename(columns={"localized_name":"hero_name"},inplace=True)
heroes_df.head()


,id,hero_name
0,1,Anti-Mage
1,2,Axe
2,3,Bane
3,4,Bloodseeker
4,5,Crystal Maiden


In [428]:
# define players df
ti_teams_df= pd.read_json('teams.json')
ti_teams = list(ti_teams_df['team_id'])
players_df = pd.read_json('proPlayers.json')
players_df = players_df[['account_id','personaname','name','team_id','team_name','team_tag']]
players_df = players_df[players_df['team_id'].isin(ti_teams)]
players_df.head()

,account_id,personaname,name,team_id,team_name,team_tag
17,10366616,Sneyking,Sneyking,8291895,Tundra Esports,Tundra
30,16497807,tOfu,tOfu,8599101,Gaimin Gladiators,GG
52,25907144,Let's Go,Cr1t-,39,Evil Geniuses,EG
83,38628747,Verissimus,MoonMeander,8260983,TSM FTX,TSM FTX
91,40547474,Aui_2000,Aui_2000,8291895,Tundra Esports,Tundra


In [429]:
#create pickbans df
picks_bans = matches_df.set_index('match_id').to_dict()['picks_bans']
picks = []
bans = []
for i in picks_bans:
    winner = get_winner(i)
    for j in picks_bans[i]:
        if j['is_pick']:
            pick = {}
            pick['match_id'] = i
            pick['order'] = j['order']
            pick['hero_id'] = j['hero_id']
            pick['team'] = j['team']
            if winner == j['team']:
                pick['win'] = 1
            else:
                pick['win'] = 0
            picks.append(pick)
        elif not j['is_pick']:
            ban = {}
            ban['match_id'] = i
            ban['order'] = j['order']
            ban['hero_id'] = j['hero_id']
            ban['team'] = j['team']
            bans.append(ban)

In [430]:
picks_df = pd.DataFrame(picks)
heroes_df['pick_count'] = heroes_df['id'].map(picks_df['hero_id'].value_counts())
wins = picks_df[['hero_id','win']].groupby(['hero_id']).sum()
wins = wins.to_dict()
heroes_df['wins'] = heroes_df['id'].map(wins['win'])
heroes_df['winrate'] = round(heroes_df['wins'] / heroes_df['pick_count'] * 100,2)

bans_df = pd.DataFrame(bans)
heroes_df['ban_count'] = heroes_df['id'].map(bans_df['hero_id'].value_counts())


heroes_df.head()

,id,hero_name,pick_count,wins,winrate,ban_count
0,1,Anti-Mage,2.0,1.0,50.00,1.0
1,2,Axe,4.0,1.0,25.00,2.0
2,3,Bane,7.0,3.0,42.86,1.0
3,4,Bloodseeker,5.0,1.0,20.00,42.0
4,5,Crystal Maiden,31.0,24.0,77.42,3.0


In [431]:
#create hero stats
players = matches_df.set_index('match_id').to_dict()['players']
hero_stats = []
for i in players:
    for j in players[i]:
        hero_stat = {}
        hero_stat['match_id'] = j['match_id']
        hero_stat['account_id'] = j['account_id']
        hero_stat['hero_id'] = j['hero_id']
        hero_stat['kills'] = j['kills']
        hero_stat['deaths'] = j['deaths']
        hero_stat['assists'] = j['assists']
        hero_stat['last_hits'] = j['last_hits']
        hero_stat['xp_per_min'] = j['xp_per_min']
        hero_stat['gold_per_min'] = j['gold_per_min']
        hero_stats.append(hero_stat)
hero_stats_df = pd.DataFrame(hero_stats)
players_teams = players_df.set_index('account_id').to_dict()['team_id']
hero_stats_df['team_id'] = hero_stats_df['account_id'].map(players_teams) 

hero_stats_df.head()

,match_id,account_id,hero_id,kills,deaths,assists,last_hits,xp_per_min,gold_per_min,team_id
0,6806663153,86799300,102,0,4,7,51,361,222,7390454
1,6806663153,221666230,52,6,4,5,300,578,526,7390454
2,6806663153,87063175,137,6,4,6,178,462,378,7390454
3,6806663153,108452107,10,2,0,1,331,647,535,7390454
4,6806663153,86726887,128,2,6,10,42,301,187,7390454


In [432]:
hero_avg_stats = hero_stats_df.groupby('hero_id').mean()
hero_max_stats = hero_stats_df.groupby('hero_id').max()
hero_min_stats = hero_stats_df.groupby('hero_id').min()

heroes_df['avg_kills'] = heroes_df['id'].map(hero_avg_stats['kills'])

heroes_df['max_kills'] = heroes_df['id'].map(hero_max_stats['kills'])

heroes_df['avg_deaths'] = heroes_df['id'].map(hero_avg_stats['deaths'])

heroes_df['min_deaths'] = heroes_df['id'].map(hero_min_stats['deaths'])

heroes_df['avg_assists'] = heroes_df['id'].map(hero_avg_stats['assists'])

heroes_df['max_assists'] = heroes_df['id'].map(hero_max_stats['assists'])

heroes_df['avg_last_hits'] = heroes_df['id'].map(hero_avg_stats['last_hits'])

heroes_df['max_last_hits'] = heroes_df['id'].map(hero_max_stats['last_hits'])

heroes_df['avg_xp_per_min'] = heroes_df['id'].map(hero_avg_stats['xp_per_min'])

heroes_df['avg_gold_per_min'] = heroes_df['id'].map(hero_avg_stats['gold_per_min'])

heroes_df.head()

,id,hero_name,pick_count,wins,winrate,ban_count,avg_kills,max_kills,avg_deaths,min_deaths,avg_assists,max_assists,avg_last_hits,max_last_hits,avg_xp_per_min,avg_gold_per_min
0,1,Anti-Mage,2.0,1.0,50.00,1.0,5.500000,7.0,5.000000,4.0,9.000000,13.0,496.500000,522.0,853.500000,694.500000
1,2,Axe,4.0,1.0,25.00,2.0,4.500000,8.0,4.250000,0.0,4.500000,10.0,203.250000,236.0,544.000000,440.500000
2,3,Bane,7.0,3.0,42.86,1.0,1.142857,4.0,5.571429,2.0,12.428571,20.0,47.142857,146.0,388.857143,263.857143
3,4,Bloodseeker,5.0,1.0,20.00,42.0,13.000000,25.0,4.000000,2.0,9.800000,13.0,385.600000,573.0,666.600000,594.800000
4,5,Crystal Maiden,31.0,24.0,77.42,3.0,2.483871,5.0,5.612903,1.0,15.903226,28.0,63.483871,146.0,434.612903,302.741935


In [433]:
heroes_5_df = heroes_df[heroes_df['pick_count']>=5]

In [434]:
player_avg_stats = hero_stats_df.groupby('account_id').mean()
player_max_stats = hero_stats_df.groupby('account_id').max()
player_min_stats = hero_stats_df.groupby('account_id').min()

players_df['avg_kills'] = players_df['account_id'].map(player_avg_stats['kills'])

players_df['max_kills'] = players_df['account_id'].map(player_max_stats['kills'])

players_df['avg_deaths'] = players_df['account_id'].map(player_avg_stats['deaths'])

players_df['min_deaths'] = players_df['account_id'].map(player_min_stats['deaths'])

players_df['avg_assists'] = players_df['account_id'].map(player_avg_stats['assists'])

players_df['max_assists'] = players_df['account_id'].map(player_max_stats['assists'])

players_df['avg_last_hits'] = players_df['account_id'].map(player_avg_stats['last_hits'])

players_df['max_last_hits'] = players_df['account_id'].map(player_max_stats['last_hits'])

players_df['avg_xp_per_min'] = players_df['account_id'].map(player_avg_stats['xp_per_min'])

players_df['avg_gold_per_min'] = players_df['account_id'].map(player_avg_stats['gold_per_min'])

players_df['max_gold_per_min'] = players_df['account_id'].map(player_max_stats['gold_per_min'])

player_unique_heroes = hero_stats_df[['account_id','hero_id']].groupby('account_id').nunique()
players_df['unique_heroes'] = players_df['account_id'].map(player_unique_heroes['hero_id'])

players_df.head()

,account_id,personaname,name,team_id,team_name,team_tag,avg_kills,max_kills,avg_deaths,min_deaths,avg_assists,max_assists,avg_last_hits,max_last_hits,avg_xp_per_min,avg_gold_per_min,max_gold_per_min,unique_heroes
17,10366616,Sneyking,Sneyking,8291895,Tundra Esports,Tundra,3.000000,6.0,6.071429,2.0,16.071429,29.0,76.285714,151.0,480.785714,320.214286,412.0,6.0
30,16497807,tOfu,tOfu,8599101,Gaimin Gladiators,GG,2.750000,7.0,5.687500,1.0,13.000000,38.0,82.625000,127.0,437.125000,304.687500,395.0,9.0
52,25907144,Let's Go,Cr1t-,39,Evil Geniuses,EG,4.187500,12.0,4.437500,0.0,15.375000,23.0,49.000000,111.0,458.312500,306.187500,376.0,10.0
83,38628747,Verissimus,MoonMeander,8260983,TSM FTX,TSM FTX,3.428571,15.0,6.857143,2.0,10.285714,23.0,75.285714,202.0,414.357143,283.500000,423.0,9.0
91,40547474,Aui_2000,Aui_2000,8291895,Tundra Esports,Tundra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [435]:
ti_teams_df = ti_teams_df[['team_id','name','tag']]
teams_stats_df = hero_stats_df.groupby(['match_id','team_id']).sum()
team_avg_stats_df = teams_stats_df.groupby('team_id').mean()
team_max_stats_df = teams_stats_df.groupby('team_id').max()
team_min_stats_df = teams_stats_df.groupby('team_id').min()

ti_teams_df['avg_kills'] = ti_teams_df['team_id'].map(team_avg_stats_df['kills'])
ti_teams_df['avg_deaths'] = ti_teams_df['team_id'].map(team_avg_stats_df['deaths'])
ti_teams_df['avg_assists'] = ti_teams_df['team_id'].map(team_avg_stats_df['assists'])
ti_teams_df['max_kills'] = ti_teams_df['team_id'].map(team_max_stats_df['kills'])
ti_teams_df['min_deaths'] = ti_teams_df['team_id'].map(team_min_stats_df['deaths'])
ti_teams_df['max_assists'] = ti_teams_df['team_id'].map(team_max_stats_df['assists'])

team_unique_heroes = hero_stats_df[['team_id','hero_id']].groupby('team_id').nunique()
ti_teams_df['unique_heroes'] = ti_teams_df['team_id'].map(team_unique_heroes['hero_id'])
ti_teams_df.head()

,team_id,name,tag,avg_kills,avg_deaths,avg_assists,max_kills,min_deaths,max_assists,unique_heroes
0,8597976,Talon,TLN,23.142857,30.000000,52.142857,46,10,98,44
1,7732977,BOOM Esports,BOOM,19.687500,24.375000,47.000000,52,6,132,39
2,7390454,Soniqs,SQ,23.437500,24.812500,52.687500,44,3,112,49
3,2163,Team Liquid,Liquid,28.437500,25.000000,65.062500,48,10,104,43
4,8260983,TSM FTX,TSM FTX,19.571429,25.785714,42.285714,44,3,99,41


In [436]:
def get_matchwinner(row):
    if row['radiant_win'] == True:
        winner = 'radiant_team_id'
    else:
        winner = 'dire_team_id'
    return row[winner]

def convert(row,column):
    seconds = row[column]
    seconds = seconds % (24 * 3600)
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60
     
    return "%d:%02d:%02d" % (hour, minutes, seconds)

ti_matches_df = matches_df[['match_id','duration','radiant_win','radiant_score','radiant_team_id','dire_score','dire_team_id']]
ti_matches_df['total_score'] = ti_matches_df['radiant_score']+ti_matches_df['dire_score']

ti_matches_df['team_winner'] = ti_matches_df.apply(lambda x: get_matchwinner(x), axis=1)
ti_matches_df['team_winner'] = ti_matches_df['team_winner'].map(ti_teams_df.set_index('team_id').to_dict()['name'])
ti_matches_df['duration_minutes'] = ti_matches_df.apply(lambda x: convert(x,'duration'), axis=1)
ti_matches_df.head()

,match_id,duration,radiant_win,radiant_score,radiant_team_id,dire_score,dire_team_id,total_score,team_winner,duration_minutes
0,6806663153,2178,False,16,7390454,18,15,34,PSG.LGD,0:36:18
1,6806047576,2971,False,23,7732977,24,2586976,47,OG,0:49:31
2,6806638898,3098,False,19,7732977,28,6209804,47,Royal Never Give Up,0:51:38
3,6806648571,2250,True,33,8131728,14,8599101,47,Hokori,0:37:30
4,6806662696,1455,False,5,2586976,34,39,39,Evil Geniuses,0:24:15


In [437]:
radiant_teams = ti_matches_df[['match_id','radiant_team_id','duration']].rename(columns={"radiant_team_id":"team_id"})
dire_teams = ti_matches_df[['match_id','dire_team_id','duration']].rename(columns={"dire_team_id":"team_id"})
ti_teams_matches = pd.concat([radiant_teams,dire_teams],ignore_index=True)
ti_teams_df['avg_length'] = ti_teams_df['team_id'].map(ti_teams_matches.groupby('team_id').mean()['duration'])
ti_teams_df['avg_length_minutes'] = ti_teams_df.apply(lambda x: convert(x,'avg_length'), axis=1)

In [438]:
hero_stats_df.to_csv('game_player_stats.csv',index=False)
players_df.to_csv('player_stats.csv', index=False)
ti_teams_df.to_csv('team_stats.csv', index=False)
ti_teams_matches.to_csv('game_head_stats.csv', index=False)

In [439]:
hero_list = []
player_list = []
team_list = []

# Hero with the most picked:

In [440]:
hero_most_picked = heroes_df[heroes_df['pick_count']==heroes_df['pick_count'].max()][['hero_name','pick_count']]
print(hero_most_picked)
obj = {}
obj["title"] = "Most Picked"
obj["result"] = hero_most_picked.rename(columns={"hero_name":"name","pick_count":"value"}).to_dict('r')
hero_list.append(obj)


        hero_name  pick_count
10   Shadow Fiend        57.0
121         Marci        57.0


# Hero with the most banned:

In [441]:
hero_most_banned = heroes_df[heroes_df['ban_count']==heroes_df['ban_count'].max()][['hero_name','ban_count']]
print(hero_most_banned)
obj = {}
obj["title"] = "Most banned"
obj["result"] = hero_most_banned.rename(columns={"hero_name":"name","ban_count":"value"}).to_dict('r')
hero_list.append(obj)

   hero_name  ban_count
31    Enigma       97.0


# Hero with the highest win rate:

In [442]:
hero_max_winrate = heroes_5_df[heroes_5_df['winrate']==heroes_5_df['winrate'].max()][['hero_name','winrate']]
print(hero_max_winrate)
obj = {}
obj["title"] = "Highest Winrate"
obj["result"] = hero_max_winrate.rename(columns={"hero_name":"name","winrate":"value"}).to_dict('r')
hero_list.append(obj)

        hero_name  winrate
4  Crystal Maiden    77.42


# Hero with the highest average kill:

In [443]:
hero_max_avg_kills = heroes_5_df[heroes_5_df['avg_kills']==heroes_5_df['avg_kills'].max()][['hero_name','avg_kills']]
print(hero_max_avg_kills)
obj = {}
obj["title"] = "Highest Average Kills"
obj["result"] = hero_max_avg_kills.rename(columns={"hero_name":"name","avg_kills":"value"}).to_dict('r')
hero_list.append(obj)


     hero_name  avg_kills
3  Bloodseeker       13.0


# Hero with the highest average assist:

In [444]:
hero_max_avg_assists = heroes_5_df[heroes_5_df['avg_assists']==heroes_5_df['avg_assists'].max()][['hero_name','avg_assists']]
print(hero_max_avg_assists)
obj = {}
obj["title"] = "Highest Average Assists"
obj["result"] = hero_max_avg_assists.rename(columns={"hero_name":"name","avg_assists":"value"}).to_dict('r')
hero_list.append(obj)

        hero_name  avg_assists
105  Earth Spirit         19.2


# Hero with the lowest average death:

In [445]:
hero_min_avg_deaths = heroes_5_df[heroes_5_df['avg_deaths']==heroes_5_df['avg_deaths'].min()][['hero_name','avg_deaths']]
print(hero_min_avg_deaths)
obj = {}
obj["title"] = "Lowest Average deaths"
obj["result"] = hero_min_avg_deaths.rename(columns={"hero_name":"name","avg_deaths":"value"}).to_dict('r')
hero_list.append(obj)

      hero_name  avg_deaths
52  Lifestealer         2.0


# Hero with the highest average last hit:

In [446]:
hero_max_avg_last_hits = heroes_5_df[heroes_5_df['avg_last_hits']==heroes_5_df['avg_last_hits'].max()][['hero_name','avg_last_hits']]
print(hero_max_avg_last_hits)
obj = {}
obj["title"] = "Highest Average Last hits"
obj["result"] = hero_max_avg_last_hits.rename(columns={"hero_name":"name","avg_last_hits":"value"}).to_dict('r')
hero_list.append(obj)

     hero_name  avg_last_hits
87  Naga Siren     518.380952


# Hero with the highest average xp per minute:

In [447]:
hero_max_avg_xp_per_min = heroes_5_df[heroes_5_df['avg_xp_per_min']==heroes_5_df['avg_xp_per_min'].max()][['hero_name','avg_xp_per_min']]
print(hero_max_avg_xp_per_min)
obj = {}
obj["title"] = "Highest Average XPM"
obj["result"] = hero_max_avg_xp_per_min.rename(columns={"hero_name":"name","avg_xp_per_min":"value"}).to_dict('r')
hero_list.append(obj)

      hero_name  avg_xp_per_min
111  Arc Warden           783.0


# Hero with the most kills in a game:

In [448]:
hero_max_kills = heroes_df[heroes_df['max_kills']==heroes_df['max_kills'].max()][['hero_name','max_kills']]
print(hero_max_kills)
obj = {}
obj["title"] = "Most Kills in a game"
obj["result"] = hero_max_kills.rename(columns={"hero_name":"name","max_kills":"value"}).to_dict('r')
hero_list.append(obj)

     hero_name  max_kills
3  Bloodseeker       25.0


# Hero with the most last hits in a game:

In [449]:
hero_max_last_hits = heroes_df[heroes_df['max_last_hits']==heroes_df['max_last_hits'].max()][['hero_name','max_last_hits']]
print(hero_max_last_hits)
obj = {}
obj["title"] = "Most Last Hits in a game"
obj["result"] = hero_max_last_hits.rename(columns={"hero_name":"name","max_last_hits":"value"}).to_dict('r')
hero_list.append(obj)

     hero_name  max_last_hits
87  Naga Siren         1196.0


# Player with the highest kill average:

In [450]:
players_df[players_df['avg_kills']==players_df['avg_kills'].max()][['name','avg_kills']]

,name,avg_kills
1748,Lumière,9.8125


# Player with the highest assist average:

In [451]:
players_df[players_df['avg_assists']==players_df['avg_assists'].max()][['name','avg_assists']]

,name,avg_assists
656,Saksa,17.428571


# Player with the lowest death average:

In [452]:
players_df[players_df['avg_deaths']==players_df['avg_deaths'].min()][['name','avg_deaths']]

,name,avg_deaths
1056,Nightfall 凛,1.125


# Player with the highest last hit average:

In [453]:
players_df[players_df['avg_last_hits']==players_df['avg_last_hits'].max()][['name','avg_last_hits']]

,name,avg_last_hits
1539,Yuragi,451.0


# Player with the highest GPM average:

In [454]:
players_df[players_df['avg_gold_per_min']==players_df['avg_gold_per_min'].max()][['name','avg_gold_per_min']]

,name,avg_gold_per_min
1748,Lumière,703.8125


# Player with the most kills:

In [455]:
players_df[players_df['max_kills']==players_df['max_kills'].max()][['name','max_kills']]

,name,max_kills
1098,Crystallis,25.0


# Player with the most assists:

In [456]:
players_df[players_df['max_assists']==players_df['max_assists'].max()][['name','max_assists']]

,name,max_assists
1418,TIMS,39.0
1454,RodjER,39.0


# Player with the most last hits:

In [457]:
players_df[players_df['max_last_hits']==players_df['max_last_hits'].max()][['name','max_last_hits']]

,name,max_last_hits
1340,Monet,1196.0


# Player with the most GPM:

In [458]:
players_df[players_df['max_gold_per_min']==players_df['max_gold_per_min'].max()][['name','max_gold_per_min']]

,name,max_gold_per_min
1539,Yuragi,1034.0


# Player with the most different heroes:

In [459]:
players_df[players_df['unique_heroes']==players_df['unique_heroes'].max()][['name','unique_heroes']]

,name,unique_heroes
1539,Yuragi,14.0
2669,萧瑟,14.0


# Team with the most kills in a game:

In [460]:
ti_teams_df[ti_teams_df['max_kills']==ti_teams_df['max_kills'].max()][['name','max_kills']]

,name,max_kills
7,Gaimin Gladiators,54


# Team with the highest kill average:

In [461]:
ti_teams_df[ti_teams_df['avg_kills']==ti_teams_df['avg_kills'].max()][['name','avg_kills']]

,name,avg_kills
13,Evil Geniuses,28.875


# Team with the least deaths in a game:

In [462]:
ti_teams_df[ti_teams_df['min_deaths']==ti_teams_df['min_deaths'].min()][['name','min_deaths']]

,name,min_deaths
2,Soniqs,3
4,TSM FTX,3
15,Team Aster,3


# Team with the most assists in a game:

In [463]:
team_most_assists = ti_teams_df[ti_teams_df['max_assists']==ti_teams_df['max_assists'].max()][['name','max_assists']]
team_most_assists

,name,max_assists
8,BetBoom Team,142


# Team who wins the longest game:

In [464]:
ti_matches_df[ti_matches_df['duration']==ti_matches_df['duration'].max()][['team_winner','duration_minutes']]

,team_winner,duration_minutes
102,BetBoom Team,1:14:33


# Team who wins the shortest game:

In [465]:
ti_matches_df[ti_matches_df['duration']==ti_matches_df['duration'].min()][['team_winner','duration_minutes']]



,team_winner,duration_minutes
120,Evil Geniuses,0:17:51


# Team with the highest length average:

In [466]:
ti_teams_df[ti_teams_df['avg_length']==ti_teams_df['avg_length'].max()][['name','avg_length_minutes']]

,name,avg_length_minutes
14,Fnatic,0:44:27


# Team with the most different heroes:

In [467]:
ti_teams_df[ti_teams_df['unique_heroes']==ti_teams_df['unique_heroes'].max()][['name','unique_heroes']]

,name,unique_heroes
5,PSG.LGD,50


# Team with the fewest different heroes:

In [468]:
ti_teams_df[ti_teams_df['unique_heroes']==ti_teams_df['unique_heroes'].min()][['name','unique_heroes']]

,name,unique_heroes
11,Entity,30


In [469]:
df = pd.DataFrame(hero_list)
df.to_json('hero_list.json',orient='records')